라이브러리

In [ ]:
!pip install xgboost
!pip install catboost
!pip install optuna
!pip install pycaret
!pip install mxnet

In [4]:
import pandas as pd
import numpy as np

import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import VotingRegressor

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import make_scorer, mean_absolute_error
import optuna
from optuna import Trial, visualization
import time
from pycaret.regression import *

데이터로드

In [23]:
# 원본데이터
train = pd.read_csv('/content/drive/MyDrive/surface_tp_train.csv')
test=pd.read_csv('/content/drive/MyDrive/surface_tp_test.csv')
train = train.iloc[:,1:]
test = test.iloc[:,1:]

# 컬럼명 정리
col = ['stn','year','mmddhh','ta','td','hm','ws','rn','re','ww','ts','si','ss','sn']
col2 = ['stn','year','mmddhh','ta','td','hm','ws','rn','re','ww','si','ss','sn']
train.columns = col
test.columns = col2

# 변경사항 수정
train.loc[train['rn'].isin([1.8, 3.3, 5.3, 19.5, 30.3, 623.5]),'rn'] = -99.9
test.loc[test['rn'].isin([1.8, 3.3, 5.3, 19.5, 30.3, 623.5]),'rn'] = -99.9

전처리(사계절 분리)

In [24]:
# 날짜에 따른 사계절 분리
# train[train["mmddhh"].astype(str).str.startswith("2")]
spring_train = pd.concat(
    [
        train[train["mmddhh"].astype(str).str.startswith("2")],
        train[train["mmddhh"].astype(str).str.startswith("3")],
        train[train["mmddhh"].astype(str).str.startswith("4")],
    ]
)
summer_train = pd.concat(
    [
        train[train["mmddhh"].astype(str).str.startswith("5")],
        train[train["mmddhh"].astype(str).str.startswith("6")],
        train[train["mmddhh"].astype(str).str.startswith("7")],
    ]
)
fall_train = pd.concat(
    [
        train[train["mmddhh"].astype(str).str.startswith("8")],
        train[train["mmddhh"].astype(str).str.startswith("9")],
        train[
            train["mmddhh"].astype(str).str.startswith("10")
            & (train["mmddhh"].astype(str).str.len() == 6)
        ],
    ]
)
winter_train = pd.concat(
    [
        train[
            train["mmddhh"].astype(str).str.startswith("11")
            & (train["mmddhh"].astype(str).str.len() == 6)
        ],
        train[
            train["mmddhh"].astype(str).str.startswith("12")
            & (train["mmddhh"].astype(str).str.len() == 6)
        ],
        train[
            train["mmddhh"].astype(str).str.startswith("1")
            & (train["mmddhh"].astype(str).str.len() == 5)
        ],
    ]
)

spring_test = pd.concat(
    [
        test[test["mmddhh"].astype(str).str.startswith("2")],
        test[test["mmddhh"].astype(str).str.startswith("3")],
        test[test["mmddhh"].astype(str).str.startswith("4")],
    ]
)
summer_test = pd.concat(
    [
        test[test["mmddhh"].astype(str).str.startswith("5")],
        test[test["mmddhh"].astype(str).str.startswith("6")],
        test[test["mmddhh"].astype(str).str.startswith("7")],
    ]
)
fall_test = pd.concat(
    [
        test[test["mmddhh"].astype(str).str.startswith("8")],
        test[test["mmddhh"].astype(str).str.startswith("9")],
        test[
            test["mmddhh"].astype(str).str.startswith("10")
            & (test["mmddhh"].astype(str).str.len() == 6)
        ],
    ]
)
winter_test = pd.concat(
    [
        test[
            test["mmddhh"].astype(str).str.startswith("11")
            & (test["mmddhh"].astype(str).str.len() == 6)
        ],
        test[
            test["mmddhh"].astype(str).str.startswith("12")
            & (test["mmddhh"].astype(str).str.len() == 6)
        ],
        test[
            test["mmddhh"].astype(str).str.startswith("1")
            & (test["mmddhh"].astype(str).str.len() == 5)
        ],
    ]
)

전처리(결측치 대체)

In [7]:
# 결측치 유무 확인 코드

missing_values_2 = [-99, -99.9, -999, -99.90]

for column in train.columns:
    # 각 컬럼에서 결측치가 있는지 확인
    missing_values = train[column].isin(missing_values_2).sum()

    if missing_values > 0:
        print(f"{column}: {missing_values} missing values")
    else:
        print(f"{column}: No missing values")

stn: No missing values
year: No missing values
mmddhh: No missing values
ta: 630 missing values
td: 646 missing values
hm: 578 missing values
ws: 1007 missing values
rn: 7995 missing values
re: 2139 missing values
ww: No missing values
ts: 627 missing values
si: 200026 missing values
ss: 199063 missing values
sn: 431046 missing values


spring 계절을 기준으로 코드 작성함 나머지 다른 계절도 동일하게 진행

In [25]:
df_train = spring_train.copy()
df2 = df_train.replace([-99, -99.9, -999], np.nan)

In [26]:
# 평균,중위수,보간법으로 결측치를 대체하는 함수 V1(Q),V2(Q),V3(Q)

def V1(Q):
  df2 = df_train.replace([-99, -99.9, -999], np.nan)
  df2[Q][df2[df2[Q].isna()].index] = np.nanmean(df2[Q]) # 컬럼Q nan값 평균값 대체
  df2 = df2.fillna(0)   # 나머지 결측치 0으로 변경

  # 필요없는 컬럼들 drop
  to_drops = [
      "year"]

  # X, y로 분리
  X = df2.drop(["ts"], axis='columns')
  y = df2["ts"]

  # object => LabelEncoder, !object => StandardScaler
  object_cols = []
  non_object_cols = []
  for col in X.columns:
      if X[col].dtype == 'object':
          object_cols.append(col)
      else:
          non_object_cols.append(col)

  # LabelEncoder 적용
  le = LabelEncoder()
  for col in object_cols:
      X[col] = le.fit_transform(X[col])

  # StandardScaler 적용
  ss = StandardScaler()
  for col in non_object_cols:
      X[col] = ss.fit_transform(X[[col]])

  # lightGBM + 교차검증 + MAE(평가지표)
  # LightGBM 모델 생성
  model = LGBMRegressor(random_state=42)

  # 교차 검증 설정
  kf = KFold(n_splits=5, random_state=42, shuffle=True)
  scoring = make_scorer(mean_absolute_error, greater_is_better=True)
  cv_scores = cross_val_score(model, X, y, cv=kf, scoring=scoring)

  # 전체 교차 검증 결과 출력
  print(f"평균대체{Q}")
  print(f"전체 MAE: {cv_scores}")
  print(f"평균 MAE: {np.mean(cv_scores)}")
  print(f"MAE 표준편차: {np.std(cv_scores)}")

def V2(Q):
  df2 = df_train.replace([-99, -99.9, -999], np.nan)
  df2[Q][df2[df2[Q].isna()].index] = np.nanmedian(df2[Q])    # ta컬럼 nan값 중위수로 대체
  df2 = df2.fillna(0)   # 나머지 결측치 0으로 변경

  # @@@데이터 로드@@@ "eval(input(...))"을 입력 변수로 수정해주세요
  dfm = df2

  # 필요없는 컬럼들 drop
  to_drops = [
      "year",
  ]
  # for to_drop in to_drops:
  #     if to_drop in dfm.columns:
  #         dfm = dfm.drop(to_drop, axis='columns')

  # X, y로 분리
  X = dfm.drop(["ts"], axis='columns')
  y = dfm["ts"]

  # object => LabelEncoder, !object => StandardScaler
  object_cols = []
  non_object_cols = []
  for col in X.columns:
      if X[col].dtype == 'object':
          object_cols.append(col)
      else:
          non_object_cols.append(col)

  # LabelEncoder 적용
  le = LabelEncoder()
  for col in object_cols:
      X[col] = le.fit_transform(X[col])

  # StandardScaler 적용
  ss = StandardScaler()
  for col in non_object_cols:
      X[col] = ss.fit_transform(X[[col]])

  # lightGBM + 교차검증 + MAE(평가지표)
  # LightGBM 모델 생성
  model = LGBMRegressor(random_state=42)

  # 교차 검증 설정
  kf = KFold(n_splits=5, random_state=42, shuffle=True)
  scoring = make_scorer(mean_absolute_error, greater_is_better=True)
  cv_scores = cross_val_score(model, X, y, cv=kf, scoring=scoring)

  # 전체 교차 검증 결과 출력
  print(f"중위수대체{Q}")
  print(f"전체 MAE: {cv_scores}")
  print(f"평균 MAE: {np.mean(cv_scores)}")
  print(f"MAE 표준편차: {np.std(cv_scores)}")

def V3(Q):
  df2 = df_train.replace([-99, -99.9, -999], np.nan)
  df2[Q] = df2[Q].interpolate(method='linear', limit_direction='forward', x=df2[df2.corr()[Q].drop(Q).idxmax()])  # ta컬럼 nan값 보간법으로 대체.  df2.corr()['ta'].drop('ta').idxmax(): ta컬럼과 상관관계가 가장 높은 컬럼
  df2=df2.fillna(0)   # 나머지 결측치 0으로 변경

  # @@@데이터 로드@@@ "eval(input(...))"을 입력 변수로 수정해주세요
  dfm = df2

  # 필요없는 컬럼들 drop
  to_drops = [
      "year",
  ]
  # for to_drop in to_drops:
  #     if to_drop in dfm.columns:
  #         dfm = dfm.drop(to_drop, axis='columns')

  # X, y로 분리
  X = dfm.drop(["ts"], axis='columns')
  y = dfm["ts"]

  # object => LabelEncoder, !object => StandardScaler
  object_cols = []
  non_object_cols = []
  for col in X.columns:
      if X[col].dtype == 'object':
          object_cols.append(col)
      else:
          non_object_cols.append(col)

  # LabelEncoder 적용
  le = LabelEncoder()
  for col in object_cols:
      X[col] = le.fit_transform(X[col])

  # StandardScaler 적용
  ss = StandardScaler()
  for col in non_object_cols:
      X[col] = ss.fit_transform(X[[col]])

  # lightGBM + 교차검증 + MAE(평가지표)
  # LightGBM 모델 생성
  model = LGBMRegressor(random_state=42)

  # 교차 검증 설정
  kf = KFold(n_splits=5, random_state=42, shuffle=True)
  scoring = make_scorer(mean_absolute_error, greater_is_better=True)
  cv_scores = cross_val_score(model, X, y, cv=kf, scoring=scoring)

  # 전체 교차 검증 결과 출력
  print(f"보간법{Q}")
  print(f"전체 MAE: {cv_scores}")
  print(f"평균 MAE: {np.mean(cv_scores)}")
  print(f"MAE 표준편차: {np.std(cv_scores)}")

In [14]:
pd.options.mode.chained_assignment = None  # 경고 메시지 무시

In [ ]:
# 각 컬럼별 결측치를 대체하는 최상의 방법이 무엇인지 확인하는 for문
pd.options.mode.chained_assignment = None  # 경고 메시지 무시

col_list = ['ta','td','hm','ws','rn','re','ts','si','ss','sn']
for i in col_list:
  V1(i)
  V2(i)
  V3(i)
  print('-'*50)

피처 엔지니어링

In [27]:
# 피처 엔지니어링
df2['mm'] = df2['mmddhh'] // 10000           # 'mmddhh' 컬럼에서 'mm', 'dd', 'hh' 분리
df2['dd'] = (df2['mmddhh'] // 100) % 100
df2['hh'] = df2['mmddhh'] % 100

계절별 train, test

In [28]:
# spring_train 만들기(다른 계절도 동일)

# spring_train
df2 = df_train.replace([-99, -99.9, -999], np.nan)
df2['ta'][df2[df2['ta'].isna()].index] = np.nanmean(df2['ta'])
df2['td'] = df2['td'].interpolate(method='linear', limit_direction='forward', x=df2[df2.corr()['td'].drop('td').idxmax()])
df2['hm'][df2[df2['hm'].isna()].index] = np.nanmedian(df2['hm'])
df2['ws'] = df2['ws'].interpolate(method='linear', limit_direction='forward', x=df2[df2.corr()['ws'].drop('ws').idxmax()])
df2['rn'][df2[df2['rn'].isna()].index] = np.nanmean(df2['rn'])
df2['re'] = df2['re'].interpolate(method='linear', limit_direction='forward', x=df2[df2.corr()['re'].drop('re').idxmax()])
df2['ts'] = df2['ts'].interpolate(method='linear', limit_direction='forward', x=df2[df2.corr()['ts'].drop('ts').idxmax()])
df2['si'] = df2['si'].interpolate(method='linear', limit_direction='forward', x=df2[df2.corr()['si'].drop('si').idxmax()])
df2['ss'] = df2['ss'].interpolate(method='linear', limit_direction='forward', x=df2[df2.corr()['ss'].drop('ss').idxmax()])
df2['sn'][df2[df2['sn'].isna()].index] = np.nanmedian(df2['sn'])
df2=df2.fillna(0)

df2['mm'] = df2['mmddhh'] // 10000           # 'mmddhh' 컬럼에서 'mm', 'dd', 'hh' 분리
df2['dd'] = (df2['mmddhh'] // 100) % 100
df2['hh'] = df2['mmddhh'] % 100

In [29]:
df2.to_csv('Feature Engineering(spring_train).csv',index=False)
df_spring_train = df2

In [30]:
# spring_test 만들기 (다른 계절도 동일)

df_test = spring_test.copy()
df_train['test_ta']=df_test['ta']

# spring_train의 각 컬럼별 선택한 방법을 spring_test의 각 컬럼에도 동일하게 적용시켜주는 함수

def test(mean,median,bogan): # 변수는 컬럼 리스트로 넣을 것
  '''train에서 평균법이 최상의방법이었던 컬럼명은 첫번째 리스트로, 중위수법이 최상이었던 컬럼명은 두번째 리스트로, 보간법이 최상이었던 컬럼명은 세번째 리스트로
  예시::test(['ta','td','sn'],[],['hm','ws','rn','re','si','ss']  '''
  df = df_test.copy()
  dft = df_train.copy()
  df2 = df.replace([-99, -99.9, -999], np.nan)  # 결측치값들 nan으로 변환
  dft2 = dft.replace([-99, -99.9, -999], np.nan)

  for i in mean: # 평균법
    df2[i][df2[df2[i].isna()].index] = np.nanmean(dft2[i])

  for i in median: # 중위수
    df2[i][df2[df2[i].isna()].index] = np.nanmedian(dft2[i])

  list_bo1 = bogan # 보간법
  list_bo2 = [i+'_test' for i in list_bo1]
  for i, j in zip(list_bo2,list_bo1):
    dft2[i] = df2[j]
    df2[j] = dft2[i].interpolate(method='linear', limit_direction='forward', x=dft2[dft2.corr()[i].drop(i).idxmax()])
    dft2 = dft2.drop(i,axis=1)

  df2 = df2.fillna(0) # 나머지 결측치 0으로 변경

  df2['mm'] = df2['mmddhh'] // 10000           # 'mmddhh' 컬럼에서 'mm', 'dd', 'hh' 분리
  df2['dd'] = (df2['mmddhh'] // 100) % 100
  df2['hh'] = df2['mmddhh'] % 100
  return df2

df2=test(['ta','rn'],['hm','sn'],['td','ws','re','si','ss'])#봄


In [31]:
df2.to_csv('Feature Engineering(spring_test).csv',index=False)
df_spring_test = df2

전처리(columns drop, 라벨인코딩, 표준화)

In [32]:
## pre 데이터 생성
pref = df_spring_train.copy()
preg = df_spring_test.copy()

pref['stn'] = pref['stn'].astype(str)

# 1. 필요없는 컬럼들 drop
to_drops = [
    "year",
]
for to_drop in to_drops:
    if to_drop in pref.columns:
        pref = pref.drop(to_drop, axis="columns")
    if to_drop in preg.columns:
        preg = preg.drop(to_drop, axis="columns")

y = pref['ts']
pref = pref.drop('ts',axis=1)

# 2. 라벨 인코더 미리 적용, 정규화도 미리 적용하고, 변화된 부분만 다시 적용
object_cols = []
non_object_cols = []
for col in pref.columns:
    if pref[col].dtype == "object":
        object_cols.append(col)
    else:
        non_object_cols.append(col)

# LabelEncoder 적용
from sklearn.preprocessing import LabelEncoder
for col in object_cols:
    le = LabelEncoder()
    le = le.fit(pref[col])
    pref[col] = le.transform(pref[col])

    for label in np.unique(preg[col]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    preg[col] = le.transform(preg[col])

# StandardScaler 적용
from sklearn.preprocessing import StandardScaler
for col in non_object_cols:
    ss = StandardScaler()
    pref[col] = ss.fit_transform(pref[[col]])

    if col in preg.columns:
        preg[col] = ss.transform(preg[[col]])
X = pref

머신러닝 모델탐색

In [33]:
# AutoML로 적당한 모델 탐색
setup(data=X, target=y)
best_model = compare_models(sort='MAE')

,Description,Value
0,Session id,2351
1,Target,ts
2,Target type,Regression
3,Original data shape,"(107040, 16)"
4,Transformed data shape,"(107040, 16)"
5,Transformed train set shape,"(74928, 16)"
6,Transformed test set shape,"(32112, 16)"
7,Numeric features,15
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,1.1288,2.9618,1.7208,0.9658,0.2381,0.4303,19.7530
catboost,CatBoost Regressor,1.1435,2.7138,1.6473,0.9686,0.2500,0.4846,11.9800
rf,Random Forest Regressor,1.1986,3.3158,1.8206,0.9617,0.2499,0.4612,32.3390
xgboost,Extreme Gradient Boosting,1.1995,2.9823,1.7268,0.9655,0.2580,0.4955,5.9620
lightgbm,Light Gradient Boosting Machine,1.2851,3.3235,1.8229,0.9616,0.2773,0.5628,3.4190
knn,K Neighbors Regressor,1.4233,4.1880,2.0463,0.9516,0.2847,0.5134,3.9240
gbr,Gradient Boosting Regressor,1.5069,4.6337,2.1524,0.9464,0.3175,0.6725,9.3840
dt,Decision Tree Regressor,1.6801,6.9081,2.6280,0.9201,0.3377,0.5718,0.9060
huber,Huber Regressor,1.9114,7.4807,2.7350,0.9135,0.4032,1.0765,0.5400
br,Bayesian Ridge,1.9271,7.3957,2.7194,0.9145,0.4089,1.0973,0.0770


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [54]:
# 상위 5개 모델 직접 확인 LightGBM ExtraTreesRegressor CatBoostRegressor xgboost RandomForestRegressor
x_train,x_test,y_train,y_test =train_test_split(X,y,random_state=42)

# 모델 + 교차검증 + MAE(평가지표)

# LightGBM 모델 생성
model = LGBMRegressor(random_state=42)
# 모델 학습
model.fit(x_train, y_train)

# 교차 검증 설정
kf = KFold(n_splits=5, random_state=42, shuffle=True)
scoring = make_scorer(mean_absolute_error, greater_is_better=True)
cv_scores = cross_val_score(model, x_train, y_train, cv=kf,scoring=scoring)

# 전체 교차 검증 결과 출력
print('LGBMRegressor')
print(f"전체 MAE: {cv_scores}")
print(f"평균 MAE: {np.mean(cv_scores)}")
print(f"MAE 표준편차: {np.std(cv_scores)}")


#ExtraTreesRegressor
model = ExtraTreesRegressor(n_estimators=100, max_features="auto", random_state=42)
model.fit(x_train, y_train)
# 교차 검증 설정
kf = KFold(n_splits=5, random_state=42, shuffle=True)
scoring = make_scorer(mean_absolute_error, greater_is_better=True)
cv_scores = cross_val_score(model, x_train, y_train, cv=kf,scoring=scoring)

# 전체 교차 검증 결과 출력
print('ExtraTreesRegressor')
print(f"전체 MAE: {cv_scores}")
print(f"평균 MAE: {np.mean(cv_scores)}")
print(f"MAE 표준편차: {np.std(cv_scores)}")

# CatBoostRegressor
model = CatBoostRegressor(random_state=42,verbose=0)
model.fit(x_train, y_train)
# 교차 검증 설정
kf = KFold(n_splits=5, random_state=42, shuffle=True)
scoring = make_scorer(mean_absolute_error, greater_is_better=True)
cv_scores = cross_val_score(model, x_train, y_train, cv=kf,scoring=scoring)

# 전체 교차 검증 결과 출력
print('CatBoostRegressor')
print(f"전체 MAE: {cv_scores}")
print(f"평균 MAE: {np.mean(cv_scores)}")
print(f"MAE 표준편차: {np.std(cv_scores)}")

# xgboost
model = xgb.XGBRegressor(random_state=42)
model.fit(x_train, y_train)
# 교차 검증 설정
kf = KFold(n_splits=5, random_state=42, shuffle=True)
scoring = make_scorer(mean_absolute_error, greater_is_better=True)
cv_scores = cross_val_score(model, x_train, y_train, cv=kf,scoring=scoring)

# 전체 교차 검증 결과 출력
print('XGBRegressor')
print(f"전체 MAE: {cv_scores}")
print(f"평균 MAE: {np.mean(cv_scores)}")
print(f"MAE 표준편차: {np.std(cv_scores)}")


# RandomForestRegressor
model = RandomForestRegressor(random_state=42)
model.fit(x_train, y_train)
# 교차 검증 설정
kf = KFold(n_splits=5, random_state=42, shuffle=True)
scoring = make_scorer(mean_absolute_error, greater_is_better=True)
cv_scores = cross_val_score(model, x_train, y_train, cv=kf,scoring=scoring)

# 전체 교차 검증 결과 출력
print('RandomForestRegressor')
print(f"전체 MAE: {cv_scores}")
print(f"평균 MAE: {np.mean(cv_scores)}")
print(f"MAE 표준편차: {np.std(cv_scores)}")

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004673 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2027
[LightGBM] [Info] Number of data points in the train set: 80280, number of used features: 15
[LightGBM] [Info] Start training from score 9.289563
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013883 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1997
[LightGBM] [Info] Number of data points in the train set: 64224, number of used features: 15
[LightGBM] [Info] Start training from score 9.269720
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003746 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is 

In [55]:
# 2개 모델 앙상블

# CatBoostRegressor 모델 초기화
catboost_model = CatBoostRegressor(random_state=42,verbose=0)

# ExtraTreesRegressor 모델 초기화
extra_trees_model = ExtraTreesRegressor(random_state=42)

# VotingRegressor를 사용하여 앙상블 모델 초기화
ensemble_model = VotingRegressor([('catboost', catboost_model), ('extra_trees', extra_trees_model)])

# 모델 학습
ensemble_model.fit(x_train, y_train)

# 교차 검증 설정
kf = KFold(n_splits=5, random_state=42, shuffle=True)
scoring = make_scorer(mean_absolute_error, greater_is_better=True)
cv_scores = cross_val_score(ensemble_model, x_train, y_train, cv=kf,scoring=scoring)

# 전체 교차 검증 결과 출력
print('ensemble_model_CatBoostRegressor_ExtraTreesRegressor')
print(f"전체 MAE: {cv_scores}")
print(f"평균 MAE: {np.mean(cv_scores)}")
print(f"MAE 표준편차: {np.std(cv_scores)}")

ensemble_model_CatBoostRegressor_ExtraTreesRegressor
전체 MAE: [1.11382535 1.10456839 1.10740295 1.08722261 1.09979181]
평균 MAE: 1.1025622207316905
MAE 표준편차: 0.008913972123421461


In [60]:
# 3개 모델 앙상블

# CatBoostRegressor 모델 초기화
catboost_model = CatBoostRegressor(random_state=42,verbose=0)

# XGBoost 모델 초기화
xgboost_model = xgb.XGBRegressor(random_state=42)

# ExtraTreesRegressor 모델 초기화
ExtraTreesRegressor_model = ExtraTreesRegressor(random_state=42)

# VotingRegressor를 사용하여 앙상블 모델 초기화
ensemble_model_3 = VotingRegressor([('catboost', catboost_model), ('ExtraTrees', ExtraTreesRegressor_model), ('xgboost', xgboost_model)])
# 모델 학습
ensemble_model_3.fit(x_train, y_train)

# 교차 검증 설정
kf = KFold(n_splits=5, random_state=42, shuffle=True)
scoring = make_scorer(mean_absolute_error, greater_is_better=True)
cv_scores = cross_val_score(ensemble_model_3, x_train, y_train, cv=kf,scoring=scoring)

# 전체 교차 검증 결과 출력
print('ensemble_model_CatBoostRegressor_ExtraTreesRegressor_RandomForestRegressor')
print(f"전체 MAE: {cv_scores}")
print(f"평균 MAE: {np.mean(cv_scores)}")
print(f"MAE 표준편차: {np.std(cv_scores)}")

# 따라서 2개 모델만 앙상블 함.

ensemble_model_CatBoostRegressor_ExtraTreesRegressor_RandomForestRegressor
전체 MAE: [1.12288349 1.10988086 1.11026542 1.09321189 1.10632061]
평균 MAE: 1.1085124555361656
MAE 표준편차: 0.009490876119451411


In [47]:
pf = PolynomialFeatures(degree=2)
X_poly = pf.fit_transform(X)
# CatBoostRegressor 모델 초기화
catboost_model = CatBoostRegressor(random_state=42,verbose=0)

# ExtraTreesRegressor 모델 초기화
extra_trees_model = ExtraTreesRegressor(random_state=42)

# VotingRegressor를 사용하여 앙상블 모델 초기화
ensemble_model = VotingRegressor([('catboost', catboost_model), ('extra_trees', extra_trees_model)])
ensemble_model.fit(X_poly,y)

preg_poly = pf.transform(preg)
y_pred = ensemble_model.predict(preg_poly)
print(y_pred)
print(len(preg),len(df_spring_test))

# 답 csv파일로 저장
pd.concat([spring_test, pd.DataFrame(y_pred)], axis="columns").to_csv("output_spring.csv")

[ 0.32020952 -0.09246969 -0.23210917 ... 14.12706447 13.90359684
 13.55316244]
6408 6408


In [ ]:
# optuna 파라메터튜닝 + LGBM 모델

In [ ]:
# Objective 함수 정의
def objective(trial):
    # 하이퍼파라미터 탐색 범위 설정
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 10, 200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    # LGBMRegressor 모델 초기화
    model = LGBMRegressor(random_state=42, **params)

    # 모델 학습
    model.fit(x_train, y_train)

    # 평가 지표 (MAE) 계산
    mae = mean_absolute_error(y_test, model.predict(x_test))

    return mae

# 데이터 분할
# x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Optuna 최적화 설정
study = optuna.create_study(direction='minimize')

# 최적화 진행 상황 출력
def print_study_statistics(study, trial):
    print(f'Trial {trial.number}/{len(study.trials)} - Loss: {trial.value:.4f} - Best Loss: {study.best_value:.4f} - Params: {trial.params}')

study.optimize(objective, n_trials=100, callbacks=[print_study_statistics])

# 최적 하이퍼파라미터 출력
print('Best Parameters:', study.best_params)
print('Best MAE:', study.best_value)

# # 최적 하이퍼파라미터로 모델 재학습
# best_params = study.best_params
# model = LGBMRegressor(random_state=42, **best_params)
# model.fit(x_train, y_train)

# # 모델 성능 평가 - 교차 검증
# kf = KFold(n_splits=5, random_state=42, shuffle=True)
# scoring = make_scorer(mean_absolute_error, greater_is_better=True)
# cv_scores = cross_val_score(model, X, y, cv=kf,scoring=scoring)



In [56]:
# optuna 파라메터튜닝 + LGBM
pf = PolynomialFeatures(degree=2)
x_train_poly = pf.fit_transform(x_train)
model = LGBMRegressor(random_state=42,num_leaves=197,learning_rate=0.2237698754656206,max_depth=10,min_child_samples=5)
model.fit(x_train_poly,y_train)
# 교차검증

kf = KFold(n_splits=5, random_state=42, shuffle=True)
scoring = make_scorer(mean_absolute_error, greater_is_better=True)
cv_scores = cross_val_score(model, x_train_poly, y_train, cv=kf,scoring=scoring)

# 전체 교차 검증 결과 출력
print('optuna 파라메터튜닝 + LGBM')
print(f"전체 MAE: {cv_scores}")
print(f"평균 MAE: {np.mean(cv_scores)}")
print(f"MAE 표준편차: {np.std(cv_scores)}")
# 기존의 앙상블 모델보다 더 좋은 성능을 냄.
# 그러나 실제 제출시엔 앙상블 모델이 더 좋은 결과를 냈음.

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.093642 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 28334
[LightGBM] [Info] Number of data points in the train set: 80280, number of used features: 135
[LightGBM] [Info] Start training from score 9.289563
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.122451 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 28177
[Li

In [57]:
pf = PolynomialFeatures(degree=2)
X_poly = pf.fit_transform(X)
model = LGBMRegressor(random_state=42,num_leaves=197,learning_rate=0.2237698754656206,max_depth=10,min_child_samples=5)
model.fit(X_poly,y)

preg_poly =pf.transform(preg)
y_pred = model.predict(preg_poly)
print(y_pred)
print(len(preg),len(df_spring_test))

# 답 csv파일로 저장
pd.concat([spring_test, pd.DataFrame(y_pred)], axis="columns").to_csv("output_spring.csv")

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.171754 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 28498
[LightGBM] [Info] Number of data points in the train set: 107040, number of used features: 135
[LightGBM] [Info] Start training from score 9.274787
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[ 0.63145741  1.78320302  0.58273773 ... 15.31616844 14.27190792
 14.00012022]
6408 6408
